In [13]:
import torch
import numpy as np
from PIL import Image, ImageDraw
from torchvision.transforms.functional import to_pil_image
from facenet_pytorch.models.mtcnn import PNet, RNet, ONet  # Import P-Net, R-Net, O-Net
from facenet_pytorch import MTCNN
import os
#import matplotlib.pyplot as plt

# 1. Define fine-tuned P-Net, R-Net, and O-Net for finetuning
class FinetunedMTCNN(MTCNN):
#(self, image_size=160, margin=5, **kwargs): # use this after some time to improve the final results
    def __init__(self, image_size=160, margin=15, **kwargs):
        super(FinetunedMTCNN, self).__init__(**kwargs)

        # Create your custom, finetuned P-Net, R-Net, O-Net here
        self.pnet = PNet()
        self.rnet = RNet()
        self.onet = ONet()


    def forward(self, x):
        # Overriding forward pass if additional finetuning is needed
        return super().forward(x)



#new
finetuned_mtcnn = FinetunedMTCNN(keep_all=True, device='cuda:0' if torch.cuda.is_available() else 'cpu', min_face_size=60, thresholds=[0.99, 0.99, 0.99])
#new
def find_euclidean_distance(src, dst):
    return np.linalg.norm(src - dst)

def alignment_procedure(img, left_eye, right_eye):    
    left_eye_x, left_eye_y = left_eye
    right_eye_x, right_eye_y = right_eye
    
    # Find the direction to rotate the image based on the eye coordinates
    if left_eye_y > right_eye_y:
        point_3rd = (right_eye_x, left_eye_y)
        direction = -1  # Clockwise
    else:
        point_3rd = (left_eye_x, right_eye_y)
        direction = 1  # Counter-clockwise
    
    # Calculate the length of the triangle edges
    a = find_euclidean_distance(np.array(left_eye), np.array(point_3rd))
    b = find_euclidean_distance(np.array(right_eye), np.array(point_3rd))
    c = find_euclidean_distance(np.array(left_eye), np.array(right_eye))
    
    # Apply cosine rule to find the angle
    if b != 0 and c != 0:  # Avoid division by zero
        cos_a = (b**2 + c**2 - a**2) / (2 * b * c)
        angle = np.arccos(cos_a)  # Angle in radians
        angle = np.degrees(angle)  # Convert to degrees
        
        # Adjust the angle based on the rotation direction
        if direction == -1:
            angle = 90 - angle
        
        # Rotate the image using PIL
        #img = Image.fromarray(img)
        img = img.rotate(direction * angle, resample=Image.BICUBIC)
        img = np.array(img)  # Convert back to numpy array
        # aligned_img_pil = Image.fromarray(img)  # Convert back to PIL Image for saving
        # aligned_img_pil.show()
    return img
# Function to perform face detection and store image paths with their cropped face regions

def process_and_save_match_faces(image_folder, output_folder, mtcnn_model, margin=0):
    # global match_photos_all
    # match_photos_all = []
    match_photos_paths = []
    for filename in os.listdir(image_folder):
        if filename.endswith(".jpeg"):
            image_path = os.path.join(image_folder, filename)
            image = Image.open(image_path)
            boxes, confidences, landmarks = mtcnn_model.detect(image, landmarks=True)
            if boxes is not None:
                # Set a confidence threshold
                threshold = 0.95
                # Filter detected faces based on the confidence score
                filtered_faces = [i for i, confidence in enumerate(confidences) if confidence > threshold]
                #print(f"Number of filtered faces: {len(filtered_faces)}")
                # Process each filtered face
                for i in filtered_faces:
                    box = boxes[i]  # Get the bounding box for the filtered face
                    box = [int(b) for b in box]  # Ensure the box is in integer format
                    # adding margin around the box
                    # Apply margin to the bounding box
                    x1 = max(0, box[0] - margin)  # Left
                    y1 = max(0, box[1] - margin)  # Top
                    x2 = min(image.width, box[2] + margin)  # Right
                    y2 = min(image.height, box[3] + margin)  # Bottom
                    # Crop the face from the image
                    cropped_face = image.crop((x1, y1, x2, y2))
                    if cropped_face is not None:
                        # Get the landmarks (left and right eyes) for the current face
                        left_eye = landmarks[i][0]  # Left eye coordinates for face i
                        right_eye = landmarks[i][1]  # Right eye coordinates for face i
                        # Align the cropped face using the eye coordinates
                        aligned_face = alignment_procedure(cropped_face, left_eye, right_eye)
                        final_face = Image.fromarray(aligned_face)
                        face_output_path = os.path.join(output_folder, f'face_{filename}_{i}.jpeg')
                        final_face.save(face_output_path)
                        #photos.append(cropped_face)
                        # Store the aligned face and the original image path
                        #match_photos_all.append(aligned_face)
                        #match_photos_paths.append(image_path)  # Store original image path
    return match_photos_paths  # Return list of original image paths

# # 2. Function to perform face detection, apply NMS, and save cropped face images
# def process_and_save_faces(image_folder, output_folder, mtcnn_model):
#     global photos
#     photos = []
#     for filename in os.listdir(image_folder):
#         if filename.endswith(".jpeg"):
#             image_path = os.path.join(image_folder, filename)
#             image = Image.open(image_path)

#             # Detect faces (with bounding boxes, probabilities, and landmarks)
#             boxes, probs, landmarks = mtcnn_model.detect(image, landmarks=True)
#             #print(f"box{boxes} prob{probs} landmarks{landmarks}")
#             #print(boxes)
#             #print(probs)
#             #print(landmarks)
#             if boxes is not None:
#                 # Set a confidence threshold
#                 threshold = 0.96

#                 # Filter detected faces based on the confidence score
#                 filtered_faces = [i for i, prob in enumerate(probs) if prob > threshold]

#                 # Save cropped faces that pass the confidence threshold
#                 for i in filtered_faces:
#                     box = boxes[i]  # Get the bounding box for the filtered face
#                     # Crop the face using the O-Net bounding box
#                     cropped_face = image.crop(box)

#                     # Save the cropped face image in the output folder
#                     face_output_path = os.path.join(output_folder, f'face_{filename}_{i}.jpeg')
#                     cropped_face.save(face_output_path)
#                     photos.append(cropped_face)
#                     # Optionally visualize the stages
#                     #visualize_detection_stages(image, boxes, landmarks, f'Stage {i+1}')
#     print(f"number of match faces detected = {len(photos)}")

match_images_folder = r'D:\Images\Facial_reco_images\Facial_reco_images\Game'  #match photos
output_folder_match = r'D:\Images\Facial_reco_images\Facial_reco_images\Maxi'  #saving match photos
process_and_save_match_faces(match_images_folder, output_folder_match, finetuned_mtcnn, margin=50)

# studio_images_folder = '/content/drive/My Drive/studio_photos'   #studio photos
# output_folder_studio = '/content/drive/My Drive/output_studio'  #saving studio photos

# Create the output directories if they don't exist
# os.makedirs(output_folder_match, exist_ok=True)  # Create output_folder_match
# os.makedirs(output_folder_studio, exist_ok=True)

# 4. Process both match and studio images
#process_and_save_faces(match_images_folder, finetuned_mtcnn)
#process_and_save_faces(studio_images_folder, output_folder_studio, finetuned_mtcnn)






[]